In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

model_path = 'results\checkpoint-1000'
model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model.eval()

def predict_paraphrase(sentence1, sentence2):

    inputs = tokenizer.encode_plus(
        sentence1,
        sentence2,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids = inputs['input_ids'].to(model.device)
    attention_mask = inputs['attention_mask'].to(model.device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    prediction = torch.argmax(probabilities, dim=-1).cpu().numpy()[0]
    paraphrase_mapping = {0: 'not a paraphrase', 1: 'paraphrase'}
    predicted_paraphrase = paraphrase_mapping[prediction]

    print(f"Sentence 1: {sentence1}")
    print(f"Sentence 2: {sentence2}")
    print(f"Prediction: {predicted_paraphrase}")
    print(f"Probability of 'not a paraphrase': {probabilities[0][0].item():.4f}")
    print(f"Probability of 'paraphrase': {probabilities[0][1].item():.4f}")

    return predicted_paraphrase, probabilities.cpu().numpy()

sentence1 = "Apple was founded as Apple Computer Company on April 1, 1976, to produce and market Steve Wozniak's Apple I personal computer. The company was incorporated by Wozniak and Steve Jobs in 1977. Its second computer, the Apple II, became a best seller as one of the first mass-produced microcomputers. Apple introduced the Lisa in 1983 and the Macintosh in 1984, as some of the first computers to use a graphical user interface and a mouse. By 1985, the company's internal problems included the high cost of its products and power struggles between executives. That year Jobs left Apple to form NeXT, Inc., and Wozniak withdrew to other ventures. The market for personal computers expanded and evolved throughout the 1990s, and Apple lost considerable market share to the lower-priced Wintel duopoly of the Microsoft Windows operating system on Intel-powered PC clones."
sentence2 = "Apple was established as the Apple Computer Company on April 1, 1976, with the aim of producing and marketing Steve Wozniak's Apple I personal computer. Wozniak and Steve Jobs incorporated the company in 1977, and its second computer, the Apple II, became a bestseller, marking one of the earliest mass-produced microcomputers. Apple introduced the Lisa in 1983 and the Macintosh in 1984, pioneering the use of a graphical user interface and mouse in computing. By 1985, internal struggles plagued Apple, including the high cost of its products and power struggles among executives. That year, Jobs left Apple to establish NeXT, Inc., while Wozniak ventured into other projects. Throughout the 1990s, the personal computer market expanded and evolved, with Apple losing significant market share to the Wintel duopoly, comprising Microsoft Windows on Intel-powered PC clones."
predicted_paraphrase, probabilities = predict_paraphrase(sentence1, sentence2)

Sentence 1: Significance tests generally compare the value of the population parameter in the null ypothesis to the value of the estimate from the sample.
Sentence 2: Significance tests typically compare the population parameter value in the null hypothesis with the estimated value derived from the sample.
Prediction: paraphrase
Probability of 'not a paraphrase': 0.1064
Probability of 'paraphrase': 0.8936
